In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from collections import OrderedDict

In [2]:
from datetime import datetime
from datetime import date

In [3]:
raw_df = pd.read_csv('data/twitter_user_data.csv', encoding = "ISO-8859-1")
# raw_df

# Scheme 1: Choosing numerical features as predictors and develop KNN classifier from scratch (without sklearn library)

In [6]:
raw_df.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'gender', 'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created', 'description', 'fav_number',
       'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
       'retweet_count', 'sidebar_color', 'text', 'tweet_coord', 'tweet_count',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [171]:
#choose the numerical variables
df1 = raw_df[['created', '_last_judgment_at', 'fav_number', 'tweet_count','retweet_count', 'gender', 'gender:confidence' ]]
#infer the days active from the created date and judged date
df1['days_active'] = pd.to_datetime(df1['_last_judgment_at']) - pd.to_datetime(df1['created'])
df1['days_active'] = pd.to_numeric(df1['days_active'])/(60*60*24*1000000000)
#filter df by gender confidence level and days active
df1 = df1[df1['gender:confidence']>0.7]
df1 = df1[df1['days_active']>0]

df1.describe()

/var/folders/zq/q8mjd1414v9265s8jc97phkw0000gn/T/ipykernel_96857/212958496.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['days_active'] = pd.to_datetime(df1['_last_judgment_at']) - pd.to_datetime(df1['created'])
/var/folders/zq/q8mjd1414v9265s8jc97phkw0000gn/T/ipykernel_96857/212958496.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['days_active'] = pd.to_datetime(df1['_last_judgment_at']) - pd.to_datetime(df1['created'])
/var/folders/zq/q8mjd1414v9265s8jc97phkw0000gn/T/ipykernel_96857/212958496.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

,fav_number,tweet_count,retweet_count,gender:confidence,days_active
count,13911.000000,1.391100e+04,13911.000000,13911.000000,13911.000000
mean,4231.712026,3.802764e+04,0.067716,0.999701,1224.667332
std,12152.577251,1.246919e+05,1.415388,0.009430,792.826785
min,0.000000,1.000000e+00,0.000000,0.700200,0.378472
25%,21.000000,2.219500e+03,0.000000,1.000000,511.278125
50%,527.000000,1.014800e+04,0.000000,1.000000,1208.897917
75%,3367.500000,3.719650e+04,0.000000,1.000000,1863.259722
max,341621.000000,2.680199e+06,153.000000,1.000000,3262.455556


In [172]:
#select columns to use
df1 = df1.drop(['created', '_last_judgment_at', 'gender:confidence'], axis=1)
#split dataset for training and test
test_set = df1.sample(axis=0, n=2000)
train_set = df1[~df1.index.isin(test_set.index)]
#split features and labels
y_train = train_set['gender']
x_train = train_set.drop(['gender'], axis=1)
y_test = test_set['gender']
x_test = test_set.drop(['gender'], axis=1)

Defining methods for K- Nearest Neighbor

In [170]:
def my_knn(input, df, label, metric, k=1):
    temp = df.copy()
    if metric == 'euclidean':
        temp['dist'] = euclidean_dist(temp, input)
    if metric == 'mae':
        temp['dist'] = mae(temp, input)
    if metric == 'mse':
        temp['dist'] = mse(temp, input)
    temp.sort_values(by='dist', ascending=True, inplace=True)
    df_k = temp.iloc[0:k]
    k_labels=label[df_k.index]
    lst = list(k_labels)
    prediction = max(set(lst), key=lst.count)
    return prediction

def euclidean_dist(df, input):
    to_return = []
    for n in range(len(df)):
        y = np.sqrt(np.sum(df.iloc[n] - input)**2)
        to_return.append(y)
    return to_return

def mae(df, input):
    to_return = []
    for n in range(len(df)):
        y = np.mean(abs(df.iloc[n] - input))
        to_return.append(y)
    return to_return

def mse(df, input):
    to_return = []
    for n in range(len(df)):
        y = np.mean((df.iloc[n] - input)**2)
        to_return.append(y)
    return to_return

Perform prediction on the test set


In [152]:
#Normalize input data
for c in range(len(x_test.columns)):
    n_max = np.max(pd.concat(objs=[x_train.iloc[:,c], x_test.iloc[:,c]], axis=0))
    n_min = np.min(pd.concat(objs=[x_train.iloc[:,c], x_test.iloc[:,c]], axis=0))
    x_train.iloc[:,c] = (x_train.iloc[:,c] - n_min) / (n_max - n_min)
    x_test.iloc[:,c] = (x_test.iloc[:,c] - n_min) / (n_max - n_min)

In [153]:
x_train.describe()

,fav_number,tweet_count,retweet_count,days_active
count,11911.000000,11911.000000,11911.000000,11911.000000
mean,0.012314,0.014355,0.000451,0.374918
std,0.034725,0.046780,0.009883,0.242499
min,0.000000,0.000000,0.000000,0.000000
25%,0.000059,0.000841,0.000000,0.157099
50%,0.001546,0.003853,0.000000,0.369520
75%,0.009878,0.013949,0.000000,0.569053
max,0.726697,1.000000,1.000000,1.000000


In [178]:
y_pred = []
for row in x_test.itertuples():
    pred = my_knn(input=list(row)[1:], df=x_train, label=y_train, metric='mae', k=5)
    y_pred.append(pred)
    print(pred)

male
male
female
female
male
female
brand
male
female
brand
female
female
brand
male
brand
female
female
female
male
brand
female
female
male
male
brand
female
female
female
female
male
brand
female
male
male
male
brand
male
male
female
female
male
male
brand
female
female
female
male
brand
female
female
male
female
brand
male
female
female
female
female
male
brand
female
male
female
brand
male
male
female
male
male
female
female
male
brand
female
brand
male
male
female
brand
female
male
female
male
male
female
female
brand
female
brand
female
female
female
male
female
brand
male
female
male
male
male
female
female
male
male
male
female
brand
female
male
brand
female
female
male
male
male
brand
male
female
female
female
male
female
brand
male
female
female
male
female
male
brand
brand
female
brand
male
female
brand
female
brand
brand
brand
female
male
male
female
brand
female
female
female
female
female
female
female
brand
female
brand
female
brand
brand
brand
male
female
male
female
m

In [180]:
result = pd.DataFrame(y_test)
result['pred'] = y_pred
result['true'] = result['pred'] == result['gender']
accuracy = np.sum(result['true']) / result.count(axis=0)
print("The accuracy of the prediction is : ", str(accuracy))

The accuracy of the prediction is :  gender    0.49
pred      0.49
true      0.49
dtype: float64


,fav_number,tweet_count,retweet_count,days_active
0,0,110964,0,690.900000
1,68,7471,0,1120.402083
4,37318,31462,0,558.494444
5,3901,20036,0,2055.314583
6,4122,13354,0,2741.537500
...,...,...,...,...
19993,170,4253,0,1905.266667
19994,3745,10727,0,2288.499306
19995,33834,29190,1,1005.464583
19997,1855,5640,0,194.200000
